In [1]:
import tensorflow as tf
import optuna
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, backend, constraints, initializers
from keras_self_attention import SeqSelfAttention
from sklearn.preprocessing import StandardScaler

In [2]:
!pip install keras_self_attention

In [3]:
df = pd.read_csv('/truncated_LOB_data_BTC_USD_COINBASE.csv')

FileNotFoundError: ignored

In [ ]:
def finder_of_fulfilment(df, column):
    time, indicator = [], []
    for i in range(len(df)):
        num = df[column].iloc[i]
        arr = df[df[column]>num][column].index
        time.append(pd.to_datetime(df['timestamp'].iloc[arr[arr>i][0]]) - pd.to_datetime(df['timestamp'].iloc[i])
                    if len(arr[arr>i]) else
                    pd.to_datetime('2023-10-03 00:00:00') - pd.to_datetime(df['timestamp'].iloc[i]))
        indicator.append(1 if len(arr[arr>i]) else 0)
    return time, indicator

In [ ]:
df['time'], df['indicator'] = finder_of_fulfilment(df, 'ask_prices_0')
df

In [ ]:
df['time'] = (df['time']).dt.total_seconds().astype(int)
df = df.iloc[0:-140]

In [ ]:
df_to_plot = df[['indicator', 'time']]
N = len(df_to_plot)
S_of_t = []

for time in range(max(df['time'])):
    num_executed = len(df_to_plot[df_to_plot['time']==time])
    num_survived = len(df_to_plot[df_to_plot['time']>time])
    prob = num_executed/(num_survived+num_executed)
    #print(prob)
    S_of_t.append(1-prob)

In [ ]:
S_hat = np.cumprod(S_of_t)

In [ ]:
t = np.linspace(0, int(max(df['time'])), int(max(df['time'])))

plt.figure(figsize=(6, 6))
plt.plot(t[:400], S_hat[:400], linewidth=2, label='Level 1, pegged', color='black')
plt.xlabel('t (sec.)')
plt.ylabel('$\hat{S}(t)$')
plt.title('Survival Probability Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#Make some features and drop useless columns
pd.options.mode.chained_assignment = None

df['vol_imbalance'] = (df['bid_quantity_0'] - df['ask_quantity_0'])/(df['bid_quantity_0'] + df['ask_quantity_0'])
df['microprice'] = ((df['bid_prices_0']*df['bid_quantity_0']+df['ask_prices_0']*df['ask_quantity_0'])
                    /(df['bid_quantity_0'] + df['ask_quantity_0']))

def logReturns(S):
    return np.log(S/S.shift(1))

def volFromTimeSeriesPerSecond(prices, timeStamps):
    if np.size(prices) <= 2:
        raise ValueError('We need more prices than 2')

    # Convert timestamp strings to datetime
    timeStamps = pd.to_datetime(timeStamps)

    x = logReturns(prices)[1:]  # the value at the first index is NaN
    xBar = np.mean(x)

    # Calculate deltaT in seconds
    deltaT = (timeStamps - timeStamps.shift(10)).dt.total_seconds()[1:]  # the value at the first index is NaN
    N = np.size(x)

    return np.sqrt(np.sum((x - xBar) ** 2 / deltaT) / (N - 1))

def volFromTimeSeriesPerHour(prices,timeStamps):
    return volFromTimeSeriesPerSecond(prices, timeStamps)*np.sqrt(3600.0)

def volFromTimeSeriesPerDay(prices,timeStamps):
    return volFromTimeSeriesPerSecond(prices, timeStamps)*np.sqrt(3600.0*24)

def volFromTimeSeriesPerYear(prices,timeStamps):
    return volFromTimeSeriesPerSecond(prices, timeStamps)*np.sqrt(3600.0*24*365.25)


print("Vol is %.2f%% per second" % (100.0*volFromTimeSeriesPerSecond(df['microprice'], df['timestamp'])))
print("Vol is %.2f%% per hour" % (100.0*volFromTimeSeriesPerHour(df['microprice'], df['timestamp'])))
print("Vol is %.2f%% per day" % (100.0*volFromTimeSeriesPerDay(df['microprice'], df['timestamp'])))
print("Vol is %.2f%% per year" % (100.0*volFromTimeSeriesPerYear(df['microprice'], df['timestamp'])))

In [ ]:
def calculate_volatility(prices, timeStamps):
    if len(prices) <= 1:
        return np.nan  # Not enough data

    x = logReturns(prices)
    xBar = np.mean(x)
    i=10
    deltaT = (timeStamps - timeStamps.shift(i)).dt.total_seconds()
    N = len(x)

    return np.sqrt(np.nansum((x - xBar) ** 2 / deltaT) / (N - 1))

# Define a function to calculate rolling volatility
def rolling_volatility(df, window_size):
    # Convert timestamp strings to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Apply the rolling window
    rolling_vol = df['microprice'].rolling(window=window_size).apply(
        lambda x: calculate_volatility(x, df['timestamp']), raw=False
    )

    return rolling_vol

window_size = 1000

# Calculate rolling volatility and add it as a new column
df['daily_volatility'] = rolling_volatility(df, window_size)

In [ ]:
target = [] #create target S(execution_time)
for i in range(len(df)):
    target.append(S_of_t[df['time'].iloc[i]-1])

df['target'] = target

In [ ]:
plt.figure(figsize=(10, 6))

df['timestamp'] = pd.to_datetime(df['timestamp'])

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['microprice'], color='blue', linewidth=2, label='Microprice')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Microprice')
plt.title('Microprice over Time')

# Displaying the grid
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['vol_imbalance'].rolling(1000).mean(), color='blue',
         linewidth=2, label='Rolling Volume Imbalance')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Volume Imbalance')
plt.title('Rolling Volume Imbalance over Time')

# Displaying the grid
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the microprice with a blue line
plt.plot(df['timestamp'], df['target'].rolling(1000).mean(), color='blue',
         linewidth=2, label='Target thing')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Target')
plt.title('Rolling TargetTargetTarget')

# Displaying the grid
plt.grid(True)

In [ ]:
df['daily_volatility'] = df['daily_volatility']*np.sqrt(360000.0*24)*100
df

In [ ]:
df.dropna(inplace=True)
df.describe()

In [ ]:
#df.set_index('timestamp', drop=True, inplace=True)
N = 100 # define the number of previous observation included

X = df.drop(columns=['timestamp', 'time', 'indicator', 'target'])
Y = df['indicator'].iloc[N:len(X)//10].values
print(Y.shape)

In [ ]:
#Y = Y[1:]

In [ ]:
tensor_slices = []

for i in range(N, len(X)//10):
    start_index = max(0, i - N)
    slice_df = X.iloc[start_index:i+1]

    tensor_slice = tf.convert_to_tensor(slice_df, dtype=tf.float32)
    tensor_slices.append(tensor_slice)

X = tf.stack(tensor_slices)

print(X.shape)

In [ ]:
input_shape = (N+1, N+1, 1)
encoder_input = layers.Input(shape=input_shape)

# Encoder
encoder_conv1 = layers.Conv1D(4, kernel_size=2, activation='relu', padding='casual')(encoder_input)
encoder_conv2 = layers.Conv1D(4, kernel_size=2, activation='relu', padding='casual')(encoder_conv1)
encoder_flatten = layers.Flatten()(encoder_conv2)
latent_dim = 4
encoder_output = layers.Dense(latent_dim, activation='relu')(encoder_flatten)

encoder_model = Model(encoder_input, encoder_output)

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
# Compute the shape after the last conv layer before flattening
shape_before_flattening = encoder_conv2.shape[1:]
decoder_dense1 = layers.Dense(np.prod(shape_before_flattening), activation='relu')(decoder_input)
decoder_reshape = layers.Reshape(shape_before_flattening)(decoder_dense1)
decoder_conv1 = layers.Conv2DTranspose(4, kernel_size=2, activation='relu', padding='casual')(decoder_reshape)
decoder_conv2 = layers.Conv2DTranspose(4, kernel_size=2, activation='relu', padding='casual')(decoder_conv1)
decoder_flatten = layers.Flatten()(decoder_conv2)
decoder_output = layers.Dense(1, activation='relu')(decoder_flatten)  # Adjust based on the shape of Y

decoder_model = Model(decoder_input, decoder_output)

# Autoencoder
autoencoder_input = layers.Input(shape=input_shape)
encoded = encoder_model(autoencoder_input)
decoded = decoder_model(encoded)
autoencoder_model = Model(autoencoder_input, decoded)

autoencoder_model.compile(optimizer='adam', loss='MAE')

# Reshape X to include the channel dimension
X_reshaped = np.expand_dims(X, axis=-1)

# Fit the model to X and Y
autoencoder_model.fit(X_reshaped, Y, epochs=3, batch_size=16)

# Simplified Staff (LSTM instead of Att)

In [ ]:
Y_pred = autoencoder_model.predict(X)

In [ ]:
(Y_pred)

# Some experiements (model is almost ready monotonicity change is needed or S(t) to f(t) transition)

In [ ]:
plt.plot(Y)

In [ ]:
# Define the DCC layer
# Redefine the DCC layer to include ReLU activation and causal padding

# class DilatedCausalConvolution(layers.Layer):
#     def __init__(self, filters, kernel_size, dilation_rate):
#         super().__init__()
#         self.query_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
#                                         dilation_rate=dilation_rate, padding='same', activation='tanh')
#         self.key_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
#                                       dilation_rate=dilation_rate, padding='same', activation='tanh')
#         self.value_conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
#                                         dilation_rate=dilation_rate, padding='same', activation='tanh')

#     def call(self, inputs):
#         query = self.query_conv(inputs)
#         key = self.key_conv(inputs)
#         value = self.value_conv(inputs)
#         return query, key, value

# # Define the Transformer block
# class TransformerBlock(layers.Layer):
#     def __init__(self, num_heads, d_model):
#         super().__init__()
#         self.multi_head_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)

#     def call(self, query, key, value):
#         attn_output = self.multi_head_attention(query, key, value)
#         proj_output = tf.concat(attn_output, axis=0)
#         return proj_output

# # Define the encoder using the DCC and Transformer block
# def create_encoder(input_shape, filters, kernel_size, dilation_rate, num_heads, d_model):
#     inputs = layers.Input(shape=input_shape)
#     dcc_layer = DilatedCausalConvolution(filters=filters, kernel_size=kernel_size, dilation_rate=dilation_rate)
#     query, key, value = dcc_layer(inputs)
#     transformer_block = TransformerBlock(num_heads=num_heads, d_model=d_model)
#     transformer_output = transformer_block(query, key, value)
#     model = models.Model(inputs=inputs, outputs=transformer_output)
#     return model





# new part
class DilatedCausalConvolution(layers.Layer):
    def __init__(self, filters, kernel_size, dilation_rate):
        super().__init__()
        self.conv = layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                  dilation_rate=dilation_rate, padding='causal', activation='relu')

    def call(self, inputs):
        return self.conv(inputs)

# Define the Transformer block (assuming it remains the same)
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, d_model):
        super().__init__()
        self.multi_head_attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)

    def call(self, query, key, value):
        attn_output = self.multi_head_attention(query, key, value)
        proj_output = tf.concat(attn_output, axis=0)
        return proj_output

# Define the new encoder using the updated DCC
def create_encoder(input_shape, d_model, filters, kernel_size):
    inputs = layers.Input(shape=input_shape)

    # Updated convolutional layers with specified dilation rates
    dilation_rates = [1, 2, 4, 8, 16]
    x = inputs
    for rate in dilation_rates:
        x = DilatedCausalConvolution(filters=filters, kernel_size=kernel_size, dilation_rate=rate)(x)

    normalized = layers.LayerNormalization()(x)

    # Temporal encoding (assuming a simple range-based encoding for illustration)
    temporal_encoding = tf.range(d_model, dtype=tf.float32)
    temporal_encoding = layers.Reshape((1, d_model))(temporal_encoding)
    temporal_encoding = tf.tile(temporal_encoding, [tf.shape(normalized)[0], N+1, 1])

    # Concatenating with temporal encoding
    x = tf.concat([normalized, temporal_encoding], axis=2)

    # Adjust the shape [100, 15] if necessary (e.g., using a Dense layer)
    x = layers.Dense(15)(x)

    # Multi-head self-attention
    transformer_block = TransformerBlock(num_heads=num_heads, d_model=d_model)
    transformer_output = transformer_block(x, x, x)

    model = models.Model(inputs=inputs, outputs=transformer_output)
    return model
    transformer_block = TransformerBlock(num_heads=3, d_model=d_model)
    transformer_output = transformer_block(x, x, x)

    model = models.Model(inputs=inputs, outputs=transformer_output)
    return model
# Define the monotonic decoder
def monotonic_constraint(weight_matrix):
    return tf.where(weight_matrix < 0, tf.zeros_like(weight_matrix), weight_matrix)

class CustomDense(layers.Layer):
    def __init__(self, units, activation='sigmoid', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        last_dim = input_shape[-1] if isinstance(input_shape[-1], int) else input_shape[-1].value
        self.kernel = self.add_weight(name='kernel', shape=(last_dim, self.units),
                                      initializer='zeros', constraint=monotonic_constraint, trainable=True)
        self.bias = self.add_weight(name='bias', shape=(self.units,), initializer='zeros', trainable=False)
        super().build(input_shape)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

class CustomDecoder(layers.Layer):
    def __init__(self, input_dim, units, activation='relu', num_layers=5, **kwargs):
        super().__init__(**kwargs)
        self.num_layers = num_layers
        self.dense_layers = [CustomDense(units, activation=activation) for _ in range(num_layers)]
        self.dense_layers.append(CustomDense(units, activation='sigmoid'))
        self.input_dim = input_dim

    def build(self, input_shape):
        self.dense_layers[0].build((None, self.input_dim))

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

# Combine the encoder and decoder to create the full model
def create_full_model(input_shape, filters, kernel_size, num_heads, d_model, output_shape, units):
    encoder_inputs = layers.Input(shape=input_shape)
    encoder = create_encoder(input_shape, d_model, filters, kernel_size)
    encoder_output = encoder(encoder_inputs)

    encoder_output_dim = encoder_output.shape[-1]

    decoder = CustomDecoder(input_dim=encoder_output_dim, units=units, activation='relu')
    decoder_output = decoder(encoder_output)
    flat_output = tf.keras.layers.Flatten()(decoder_output)
    final_output = tf.keras.layers.Dense(output_shape)(flat_output)
    full_model = models.Model(inputs=encoder_inputs, outputs=final_output)

    return full_model

In [ ]:
# Define model parameters
input_shape = (N+1, N+1)  # Adjust based on your actual data dimensions
num_heads = 3
output_shape = 1

In [ ]:
# Optuna objective function
def objective(trial):
    # Suggest values for the hyperparameters
    filters = trial.suggest_categorical('filters', [2, 4, 8, 16])
    units = trial.suggest_categorical('units', [1, 4, 8, 16, 32])
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5, 7])
    #dilation_rate = trial.suggest_categorical('dilation_rate', [1, 2, 3, 4])
    d_model = trial.suggest_categorical('d_model', [1, 2, 4, 8, 32])
    batch = trial.suggest_categorical('batch', [4, 8, 16, 32])

    # Create the model using the suggested values
    model = create_full_model(input_shape, filters, kernel_size, num_heads, d_model, output_shape, units)

    # Compile the model (same as your existing code)
    initial_learning_rate = 0.01
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=1000, decay_rate=0.96, staircase=True)

    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule),
                  loss=tf.keras.losses.MeanAbsoluteError())

    train_ds = tf.data.Dataset.from_tensor_slices((X, Y)).batch(batch_size = batch)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    history = model.fit(train_ds, epochs=10, callbacks=[callback], verbose=1)

    val_loss = min(history.history['loss'])
    return val_loss

# Optuna study
study = optuna.create_study(direction='minimize')  # 'minimize' if lower loss is better, 'maximize' otherwise
study.optimize(objective, n_trials=5)  # Adjust the number of trials

# Print the optimal hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
# Define model parameters
input_shape = (N+1, 23)
filters = 2
units = 32
kernel_size = 7
dilation_rate = 3
num_heads = 3
d_model = 1
output_shape = 1
batch = 4 # to determine
# Create the model
model = create_full_model(input_shape, filters, kernel_size, num_heads, d_model, output_shape, units)

# Create TensorFlow dataset
train_ds = tf.data.Dataset.from_tensor_slices((X, Y)).batch(batch_size=batch)

# Compile the model
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.96, staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.MeanAbsoluteError())

# Train the model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit(train_ds, epochs=100, callbacks=[callback])

In [ ]:
Y_pred = model.predict(X)

In [ ]:
Y_pred